In [4]:
#Import Library yang dibutuhkan

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC

In [15]:
#Import Dataset Yang Diperlukan

df = pd.read_csv('Titanic.csv', index_col=0)
df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [16]:
#Cek Status dari dataset yang di import
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [17]:
#pada data tersebut, drop data yang tidak akan dipakai
#dalam hal ini column Cabin, Name, Ticket tidak kita pakai karena datanya unique

df.drop(['Cabin','Name','Ticket'], axis=1, inplace=True)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,,
1,0,3,male,22.0,1,0,7.2500,S
2,1,1,female,38.0,1,0,71.2833,C
3,1,3,female,26.0,0,0,7.9250,S
4,1,1,female,35.0,1,0,53.1000,S
5,0,3,male,35.0,0,0,8.0500,S


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 62.6+ KB


In [19]:
#selanjutnya adalah proses imputasi pada Coloumn Embarked dan Age karena mempunyai nilai null
val = df.Embarked.mode()[0]
df['Embarked'] = df.Embarked.fillna(val)

In [20]:
val = df.Age.mean()
df['Age'] = df.Age.fillna(0)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  891 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 62.6+ KB


In [22]:
#selanjutnya adalah proses convert karena colloumn age dan Embarked tipe data nya masih object
df.Sex = df.Sex.map({'male':0, 'female':1})
df.Embarked = df.Embarked.map({'S':0, 'C':1, 'Q':2})
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    int64  
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  891 non-null    int64  
dtypes: float64(2), int64(6)
memory usage: 62.6 KB


In [23]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,,
1,0,3,0,22.0,1,0,7.2500,0
2,1,1,1,38.0,1,0,71.2833,1
3,1,3,1,26.0,0,0,7.9250,0
4,1,1,1,35.0,1,0,53.1000,0
5,0,3,0,35.0,0,0,8.0500,0


In [24]:
#define variabel x dan y untuk train dan test

feature = ['Pclass','Sex','Age','SibSp', 'Parch','Fare', 'Embarked']

x = df.loc[:, feature].values
y = df.loc[:, 'Survived'].values

In [25]:
from sklearn.preprocessing import StandardScaler
SC = StandardScaler()
x_scal = SC.fit_transform(x)

In [26]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x_scal, y, test_size=0.3, random_state=142)

In [28]:
#Logistic Regression 
LR = LogisticRegression(max_iter = 1000)
LR.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [29]:
LR.predict_proba(X_test)

array([[0.81945072, 0.18054928],
       [0.90435589, 0.09564411],
       [0.40466008, 0.59533992],
       [0.83400555, 0.16599445],
       [0.9185571 , 0.0814429 ],
       [0.83887342, 0.16112658],
       [0.67752641, 0.32247359],
       [0.91546293, 0.08453707],
       [0.41154646, 0.58845354],
       [0.84159488, 0.15840512],
       [0.20580941, 0.79419059],
       [0.90157797, 0.09842203],
       [0.12201387, 0.87798613],
       [0.88955535, 0.11044465],
       [0.22998718, 0.77001282],
       [0.54526805, 0.45473195],
       [0.84406975, 0.15593025],
       [0.58533049, 0.41466951],
       [0.05656997, 0.94343003],
       [0.90569338, 0.09430662],
       [0.88689662, 0.11310338],
       [0.89669976, 0.10330024],
       [0.77792931, 0.22207069],
       [0.95903971, 0.04096029],
       [0.84376685, 0.15623315],
       [0.84767417, 0.15232583],
       [0.05188172, 0.94811828],
       [0.84133381, 0.15866619],
       [0.25017058, 0.74982942],
       [0.89035413, 0.10964587],
       [0.

In [30]:
y_pred = LR.predict_proba(X_train)
y_pred = pd.DataFrame({'y_proba':LR.predict_proba(X_train)[:,1],'class_pred':LR.predict(X_train),'class_true':y_train})
bin = [0, 0.25, 0.5, 0.75, 1]

y_pred['group']=pd.cut(y_pred.y_proba,bins = bin)
temp = pd.pivot_table(y_pred,index='group',aggfunc='sum',values=['class_pred','class_true'])
temp

,class_pred,class_true
group,,
"(0.0, 0.25]",0,39
"(0.25, 0.5]",0,36
"(0.5, 0.75]",124,70
"(0.75, 1.0]",100,96


In [32]:
#Random Forest

RF = RandomForestClassifier(n_estimators= 1000)
RF.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [33]:


y_pred = LR.predict_proba(X_train)
y_pred = pd.DataFrame({'y_proba':RF.predict_proba(X_test)[:,1],'class_pred':RF.predict(X_test),'class_true':y_test})
bin = [0, 0.25, 0.5, 0.75, 1]

y_pred['group']=pd.cut(y_pred.y_proba,bins = bin)
temp_ = pd.pivot_table(y_pred,index='group',aggfunc='sum',values=['class_pred','class_true'])
temp_



,class_pred,class_true
group,,
"(0.0, 0.25]",0,16
"(0.25, 0.5]",0,9
"(0.5, 0.75]",31,10
"(0.75, 1.0]",70,65


In [35]:
#SVM
SV = SVC(C=0.1, probability=True)
SV.fit(X_train,y_train)

SVC(C=0.1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [36]:


y_pred = SV.predict_proba(X_train)
y_pred = pd.DataFrame({'y_proba':SV.predict_proba(X_train)[:,1],'class_pred':SV.predict(X_train),'class_true':y_train})
bin = [0, 0.25, 0.5, 0.75, 1]

y_pred['group']=pd.cut(y_pred.y_proba,bins = bin)
temp = pd.pivot_table(y_pred,index='group',aggfunc='sum',values=['class_pred','class_true'])
temp



,class_pred,class_true
group,,
"(0.0, 0.25]",0,62
"(0.25, 0.5]",0,17
"(0.5, 0.75]",33,23
"(0.75, 1.0]",166,139


In [37]:
#KNN

KN = KNeighborsClassifier(n_neighbors=10)
KN.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                     weights='uniform')

In [38]:
y_pred = KN.predict_proba(X_train)
y_pred = pd.DataFrame({'y_proba':KN.predict_proba(X_train)[:,1],'class_pred':KN.predict(X_train),'class_true':y_train})
bin = [0, 0.25, 0.5, 0.75, 1]

y_pred['group']=pd.cut(y_pred.y_proba,bins = bin)
temp = pd.pivot_table(y_pred,index='group',aggfunc='sum',values=['class_pred','class_true'])
temp

,class_pred,class_true
group,,
"(0.0, 0.25]",0,27
"(0.25, 0.5]",0,47
"(0.5, 0.75]",52,35
"(0.75, 1.0]",142,132


In [39]:
#Decision Tree
DT = DecisionTreeClassifier(max_features = 7)
DT.fit(X_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=7, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [40]:
y_pred = DT.predict_proba(X_train)
y_pred = pd.DataFrame({'y_proba':DT.predict_proba(X_test)[:,1],'class_pred':DT.predict(X_test),'class_true':y_test})
bin = [0, 0.25, 0.5, 0.75, 1]

y_pred['group']=pd.cut(y_pred.y_proba,bins = bin)
temp = pd.pivot_table(y_pred,index='group',aggfunc='sum',values=['class_pred','class_true'])
temp

,class_pred,class_true
group,,
"(0.0, 0.25]",0,0
"(0.25, 0.5]",0,0
"(0.5, 0.75]",0,0
"(0.75, 1.0]",113,75
